<a href="https://colab.research.google.com/gist/kenichi-lon/fd086ab3e9bea539d92470e48493934b/tds_qa_example_portfolio_optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import historical stock price data from the internet


First of all, Install Yahoo finance library to retrieve stock data.
We will import randomly selected 20 stocks from FTSE100 and extract the historical closing prices for the past 2 years from July 2020.

In [ ]:
# !pip install yfinance

In [1]:
# import yfinance as yf  
import numpy as np
import pandas as pd

In [2]:
# list of FTSE100 stocks available at 
# https://www.londonstockexchange.com/indices/ftse-100/constituents/table
# below are randomly selected stocks

# stocks_dict = {
#     'AAPL': 'APPLE',
#     'WMT':'WALMART',
#     'MSFT':'MICROSOFT',
#     'AAL':'AMERICAN AIRLINES GROUP'
#     }
    
# stocks = list(stocks_dict.keys()) # list of candidate stocks

# df = yf.download(stocks,'2021-01-01','2022-10-31')['Close'] # main data
# df_test = yf.download(stocks,'2022-11-01','2023-01-1')['Close'] # testing data

In [3]:
# df.to_csv("train_data.csv")
# df_test.to_csv("test_data.csv")

In [4]:
df = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [5]:
data = df.copy()
data_test = df_test.copy()

In [6]:
data = data.set_index('Date', drop=True)
data_test = data_test.set_index('Date', drop=True)

In [7]:
stocks = list(data.columns) # list of candidate stocks

In [8]:
data.head(10)

,AAL,AAPL,MSFT,WMT
Date,,,,
2021-01-04,15.130000,129.410004,217.690002,146.529999
2021-01-05,15.430000,131.009995,217.899994,145.750000
2021-01-06,15.520000,126.599998,212.250000,146.660004
2021-01-07,15.380000,130.919998,218.289993,146.649994
2021-01-08,15.130000,132.050003,219.619995,146.630005
2021-01-11,15.000000,128.979996,217.490005,147.289993
2021-01-12,15.380000,128.800003,214.929993,148.970001
2021-01-13,15.530000,130.889999,216.339996,147.449997
2021-01-14,16.440001,128.910004,213.020004,146.970001


In [9]:
# N = len(stocks) # number of candidate stocks
N = len(data.columns)
num_stocks = 2 # selected stocks
N, num_stocks

(4, 2)

In [10]:
data.describe()

,AAL,AAPL,MSFT,WMT
count,460.000000,460.000000,460.000000,460.000000
mean,18.194000,148.437957,275.351869,139.361044
std,3.281667,15.952427,31.821654,8.076548
min,11.860000,116.360001,212.250000,118.290001
25%,15.515000,135.364998,249.587494,134.134998
50%,18.245000,147.894997,275.820007,140.025002
75%,20.685000,161.110004,299.107498,144.690002
max,25.820000,182.009995,343.109985,159.869995


In [11]:
data_test.describe()

,AAL,AAPL,MSFT,WMT
count,42.000000,42.000000,42.000000,42.000000
mean,13.710000,141.859999,241.458809,146.274047
std,0.725134,7.260650,9.878504,4.541800
min,12.320000,126.040001,214.250000,138.389999
25%,13.110000,135.712498,239.002499,142.482498
50%,13.960000,143.060005,241.885002,145.334999
75%,14.185000,148.090000,247.327496,150.144997
max,14.930000,151.289993,257.220001,153.509995


# 2. Prepare data by cleansing and feature engineering


Check null values. We will drop a row with null in this example for simplicity  but please cosnider replacing the values if possible.

In [12]:
print(data.isna().sum()) # null row for each stock

AAL     0
AAPL    0
MSFT    0
WMT     0
dtype: int64


In [13]:
data_test.isna().sum() # check for test data

AAL     0
AAPL    0
MSFT    0
WMT     0
dtype: int64

In [14]:
data.dropna(inplace=True) # drop the rows
data.shape

(460, 4)

In [15]:
data_test.shape

(42, 4)

## Feature Engineering

To make prediction, we use Prophet. See more at https://facebook.github.io/prophet/

We make time series prediction based on the historical data between January 2021 and October 2022. The period to predict is November 2022 till late January 2023, which is the same period of the data in test set.

The predicted values are transformed to be the average return rate for each stock. We can use it for one of the objectives to maximise the sum of return rates.

In [ ]:
# !pip install prophet

In [16]:
from prophet import Prophet

In [39]:
pred_return_rate = []
pred_return_rate_mean = []
i = 0 # current index of stock
periods = data_test.shape[0] # number of days to forecast, same as data_test length

for i in range(N): # for each stock
    model = Prophet()
    data['ds'] = data.index # index values (dates) as ds 
    data['y'] = data.iloc[:,i] # stock close price as y
    print(data['y'])
    stock_data = data[['ds','y']] # training data
    model.fit(stock_data)
    future = model.make_future_dataframe(periods=periods) # predict 
    forecast = model.predict(future)['yhat'] # predicted values stored in yhat column
    return_rate = np.zeros(len(forecast)) # return rate to be stored in numpy array
    for j in range(len(forecast)-1): 
        return_rate[j+1] = (forecast[j+1] - forecast[j])/forecast[j] # calculate the return rate per day
    pred_return_rate.append(return_rate)
    pred_return_rate_mean.append(np.mean(return_rate)) # predicted mean return rate for the stock
    data.drop(columns=['ds','y'], inplace=True)

10:56:01 - cmdstanpy - INFO - Chain [1] start processing
10:56:01 - cmdstanpy - INFO - Chain [1] done processing


Date
2021-01-04    15.13
2021-01-05    15.43
2021-01-06    15.52
2021-01-07    15.38
2021-01-08    15.13
              ...  
2022-10-24    14.10
2022-10-25    14.29
2022-10-26    14.00
2022-10-27    13.97
2022-10-28    14.00
Name: y, Length: 460, dtype: float64


10:56:02 - cmdstanpy - INFO - Chain [1] start processing
10:56:02 - cmdstanpy - INFO - Chain [1] done processing


Date
2021-01-04    129.410004
2021-01-05    131.009995
2021-01-06    126.599998
2021-01-07    130.919998
2021-01-08    132.050003
                 ...    
2022-10-24    149.449997
2022-10-25    152.339996
2022-10-26    149.350006
2022-10-27    144.800003
2022-10-28    155.740005
Name: y, Length: 460, dtype: float64


10:56:02 - cmdstanpy - INFO - Chain [1] start processing
10:56:02 - cmdstanpy - INFO - Chain [1] done processing


Date
2021-01-04    217.690002
2021-01-05    217.899994
2021-01-06    212.250000
2021-01-07    218.289993
2021-01-08    219.619995
                 ...    
2022-10-24    247.250000
2022-10-25    250.660004
2022-10-26    231.320007
2022-10-27    226.750000
2022-10-28    235.869995
Name: y, Length: 460, dtype: float64


10:56:02 - cmdstanpy - INFO - Chain [1] start processing
10:56:02 - cmdstanpy - INFO - Chain [1] done processing


Date
2021-01-04    146.529999
2021-01-05    145.750000
2021-01-06    146.660004
2021-01-07    146.649994
2021-01-08    146.630005
                 ...    
2022-10-24    139.410004
2022-10-25    140.070007
2022-10-26    141.139999
2022-10-27    140.729996
2022-10-28    142.509995
Name: y, Length: 460, dtype: float64


In [ ]:
# print(pred_return_rate_mean)

In [ ]:
# print(np.mean(pred_return_rate_mean))

We also calculate actual return rates from the historical stock price data. We will use the results for the calculation of covariance that will be used for the second objective function to reduce the risk by diversify the portfolio items to avoid losing it altogether.

In [17]:
actual_return_rate = []
actual_return_rate_mean = []
for i in range(N): # for each stock
    stock_data = data.iloc[:,i]
    return_rate = np.zeros(len(stock_data)) # return rate to be stored in numpy array
    for j in range(len(stock_data)-1): 
        return_rate[j+1] = (stock_data[j+1] - stock_data[j])/stock_data[j] # calculate the return rate per day
    actual_return_rate.append(return_rate)
    actual_return_rate_mean.append(np.mean(return_rate)) # predicted mean return rate for the stock

In [18]:
len(actual_return_rate), len(actual_return_rate[0]), actual_return_rate_mean, actual_return_rate[0][0], actual_return_rate[1][0]

(4,
 460,
 [0.00037916552109689875,
  0.000580402761627413,
  0.00033366205028875133,
  3.8505182076294666e-05],
 0.0,
 0.0)

In [19]:
actual_return_rate_mean_mean = np.mean(actual_return_rate_mean)
actual_return_rate_mean_mean

0.00033293387877233945

Finally, we calculate the actual return rates on the test set for the final evaluation.

In [20]:
actual_return_rate_test = []
actual_return_rate_mean_test = []
for i in range(N): # for each stock
    stock_data = data_test.iloc[:,i]
    return_rate = np.zeros(len(stock_data)) # return rate to be stored in numpy array
    for j in range(len(stock_data)-1): 
        return_rate[j+1] = (stock_data[j+1] - stock_data[j])/stock_data[j] # calculate the return rate per day
    actual_return_rate_test.append(return_rate)
    actual_return_rate_mean_test.append(np.mean(return_rate)) # predicted mean return rate for the stock

In [21]:
actual_return_rate_mean_mean_test = np.mean(actual_return_rate_mean_test)
actual_return_rate_mean_mean_test

-0.0009457975679432734

#3. Formulate the cost functions and constraints to obtain a QUBO matrix


## Constraint Terms

In our portfolio optimisation example, let's assume 1 constraint: the number of stock to be included in the portfolio.

In this example, we will use PyQubo, a python library that that can conver the cost function to a quadratic unconstraintbinary optimisation matrix that can be sent to D-wave qunatum annealer and JIJ simulated quantum annealer.



In [ ]:
# !pip install pyqubo 

In [ ]:
# !pip show pyqubo

In [22]:
from pyqubo import Array, Constraint, Placeholder
import numpy as np

In [23]:
x = Array.create('x',shape=N, vartype='BINARY') # array takes binary 0 or 1 indicate to exclude or include a stock in the portfolio, which we need to optimise
x

Array([Binary('x[0]'), Binary('x[1]'), Binary('x[2]'), Binary('x[3]')])

In [24]:
# number of stocks constraint
h_const1 = (num_stocks - np.dot(x,x))**2 # MSE from the budget. This lead to x values that makes the portfolio as close as the budget
h_const1

((2.000000 + (-1.000000 * ((Binary('x[3]') * Binary('x[3]')) + (Binary('x[2]') * Binary('x[2]')) + (Binary('x[0]') * Binary('x[0]')) + (Binary('x[1]') * Binary('x[1]'))))) * (2.000000 + (-1.000000 * ((Binary('x[3]') * Binary('x[3]')) + (Binary('x[2]') * Binary('x[2]')) + (Binary('x[0]') * Binary('x[0]')) + (Binary('x[1]') * Binary('x[1]'))))))

## Cost Functions

We have set two cost functions to optimise our portfolio. One is to maximise the sum of predicted growth rates that we predicted in the feature engineering section. Another is to minimise the covariance between each of the stock items to be selected in the portfolio. We then add up two cost functions for QUBO.

In [40]:
# cost function 1 to aim the highest return
h_cost1 = 0
h_cost1 -= np.dot(x,pred_return_rate_mean) # maximisation problem. negative to make it minimisation problem

In [108]:
h_cost1

(0.000000 + (-1.000000 * ((Binary('x[3]') * -0.000074) + (Binary('x[2]') * 0.000124) + (Binary('x[0]') * -0.000510) + (Binary('x[1]') * 0.000309))))

In [26]:
# cost function 2 to balance the portfolio not to put all eggs in one basket
# minimising the sum of covariance leads to the safer choice
h_cost2 = 0
for i in range(N):
    for j in range(N):
        h_cost2 += x[i]*x[j]*np.sum((actual_return_rate[i]-actual_return_rate_mean[i])*(actual_return_rate[j]-actual_return_rate_mean[j]))/len(actual_return_rate[i])

In [134]:
h_cost2

((((Binary('x[3]') * Binary('x[3]')) * 0.089424) * 0.002174) + (((Binary('x[3]') * Binary('x[2]')) * 0.032236) * 0.002174) + (((Binary('x[3]') * Binary('x[1]')) * 0.037109) * 0.002174) + (((Binary('x[3]') * Binary('x[0]')) * 0.023093) * 0.002174) + (((Binary('x[2]') * Binary('x[3]')) * 0.032236) * 0.002174) + (((Binary('x[2]') * Binary('x[2]')) * 0.146064) * 0.002174) + (((Binary('x[2]') * Binary('x[1]')) * 0.118118) * 0.002174) + (((Binary('x[2]') * Binary('x[0]')) * 0.090517) * 0.002174) + (((Binary('x[1]') * Binary('x[3]')) * 0.037109) * 0.002174) + (((Binary('x[1]') * Binary('x[2]')) * 0.118118) * 0.002174) + (((Binary('x[1]') * Binary('x[1]')) * 0.163459) * 0.002174) + (((Binary('x[1]') * Binary('x[0]')) * 0.108028) * 0.002174) + (((Binary('x[0]') * Binary('x[3]')) * 0.023093) * 0.002174) + (((Binary('x[0]') * Binary('x[2]')) * 0.090517) * 0.002174) + (((Binary('x[0]') * Binary('x[1]')) * 0.108028) * 0.002174) + 0.000000 + (((Binary('x[0]') * Binary('x[0]')) * 0.505665) * 0.002174

In [41]:
h_cost = h_cost1 + h_cost2
# h_cost = h_cost2

In [133]:
h_cost

(((((Binary('x[3]') * Binary('x[3]')) * 0.089424) * 0.002174) + (((Binary('x[3]') * Binary('x[2]')) * 0.032236) * 0.002174) + (((Binary('x[3]') * Binary('x[1]')) * 0.037109) * 0.002174) + (((Binary('x[3]') * Binary('x[0]')) * 0.023093) * 0.002174) + (((Binary('x[2]') * Binary('x[3]')) * 0.032236) * 0.002174) + (((Binary('x[2]') * Binary('x[2]')) * 0.146064) * 0.002174) + (((Binary('x[2]') * Binary('x[1]')) * 0.118118) * 0.002174) + (((Binary('x[2]') * Binary('x[0]')) * 0.090517) * 0.002174) + (((Binary('x[1]') * Binary('x[3]')) * 0.037109) * 0.002174) + (((Binary('x[1]') * Binary('x[2]')) * 0.118118) * 0.002174) + (((Binary('x[1]') * Binary('x[1]')) * 0.163459) * 0.002174) + (((Binary('x[1]') * Binary('x[0]')) * 0.108028) * 0.002174) + (((Binary('x[0]') * Binary('x[3]')) * 0.023093) * 0.002174) + (((Binary('x[0]') * Binary('x[2]')) * 0.090517) * 0.002174) + (((Binary('x[0]') * Binary('x[1]')) * 0.108028) * 0.002174) + 0.000000 + (((Binary('x[0]') * Binary('x[0]')) * 0.505665) * 0.00217

In [136]:
text = """(((((Binary('x[3]') * Binary('x[3]')) * 0.089424) * 0.002174) + (((Binary('x[3]') * Binary('x[2]')) * 0.032236) * 0.002174) + (((Binary('x[3]') * Binary('x[1]')) * 0.037109) * 0.002174) + (((Binary('x[3]') * Binary('x[0]')) * 0.023093) * 0.002174) + (((Binary('x[2]') * Binary('x[3]')) * 0.032236) * 0.002174) + (((Binary('x[2]') * Binary('x[2]')) * 0.146064) * 0.002174) + (((Binary('x[2]') * Binary('x[1]')) * 0.118118) * 0.002174) + (((Binary('x[2]') * Binary('x[0]')) * 0.090517) * 0.002174) + (((Binary('x[1]') * Binary('x[3]')) * 0.037109) * 0.002174) + (((Binary('x[1]') * Binary('x[2]')) * 0.118118) * 0.002174) + (((Binary('x[1]') * Binary('x[1]')) * 0.163459) * 0.002174) + (((Binary('x[1]') * Binary('x[0]')) * 0.108028) * 0.002174) + (((Binary('x[0]') * Binary('x[3]')) * 0.023093) * 0.002174) + (((Binary('x[0]') * Binary('x[2]')) * 0.090517) * 0.002174) + (((Binary('x[0]') * Binary('x[1]')) * 0.108028) * 0.002174) + 0.000000 + (((Binary('x[0]') * Binary('x[0]')) * 0.505665) * 0.002174)) + 0.000000 + (-1.000000 * ((Binary('x[3]') * -0.000074) + (Binary('x[2]') * 0.000124) + (Binary('x[0]') * -0.000510) + (Binary('x[1]') * 0.000309))))"""
reg = """Binary\(.*?\)"""

In [140]:
import re
text = re.sub(r"Binary\((.*?)\)", r"\1", text)
re.sub('\'', '', text)

'(((((x[3] * x[3]) * 0.089424) * 0.002174) + (((x[3] * x[2]) * 0.032236) * 0.002174) + (((x[3] * x[1]) * 0.037109) * 0.002174) + (((x[3] * x[0]) * 0.023093) * 0.002174) + (((x[2] * x[3]) * 0.032236) * 0.002174) + (((x[2] * x[2]) * 0.146064) * 0.002174) + (((x[2] * x[1]) * 0.118118) * 0.002174) + (((x[2] * x[0]) * 0.090517) * 0.002174) + (((x[1] * x[3]) * 0.037109) * 0.002174) + (((x[1] * x[2]) * 0.118118) * 0.002174) + (((x[1] * x[1]) * 0.163459) * 0.002174) + (((x[1] * x[0]) * 0.108028) * 0.002174) + (((x[0] * x[3]) * 0.023093) * 0.002174) + (((x[0] * x[2]) * 0.090517) * 0.002174) + (((x[0] * x[1]) * 0.108028) * 0.002174) + 0.000000 + (((x[0] * x[0]) * 0.505665) * 0.002174)) + 0.000000 + (-1.000000 * ((x[3] * -0.000074) + (x[2] * 0.000124) + (x[0] * -0.000510) + (x[1] * 0.000309))))'

## Prepare QUBO

The problme formulation and representation in QUBO format are habled by PyQubo applications. In the lines below, we compile the model using the objective function that needs to be minimised, then define the constraint coefficient. to_qubo function generates a QUBO matrix in the dictionary type and an offset value which is an costanct value not required for the search for the minimum.

In [42]:
h = h_cost + Placeholder('l1')*Constraint(h_const1,label='num_stocks')
model = h.compile()

In [43]:
feed_dict = {'l1': 2}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [44]:
offset

8.0

# 4. Sample optimisation results from a D-Wave solver


Install D-Wave's Ocean SDK to request the computation to the quantum annealer. EmbeddingComposite is used to embed the QUBO to the physical quantum anneler in D-Wave.


In [ ]:
# !pip install dwave-ocean-sdk # https://docs.ocean.dwavesys.com/en/stable/

In [160]:
from dwave.system import DWaveSampler, EmbeddingComposite
endpoint = 'https://cloud.dwavesys.com/sapi'
token = 'DEV-0ed91ebd68205c97e2c6d646163cb76020f46581' # Please relace with your own token

Available annelers depend on the geographic location and machine's online status. The list of solvers can be found on the D-Wave Leap dashboard. 

In [163]:
import time

In [191]:
qubo

{('x[0]', 'x[0]'): -5.998390519092348,
 ('x[3]', 'x[1]'): 4.000161344935638,
 ('x[2]', 'x[0]'): 4.00039355311639,
 ('x[3]', 'x[0]'): 4.000100405430845,
 ('x[3]', 'x[3]'): -5.999731114980669,
 ('x[2]', 'x[1]'): 4.000513558303082,
 ('x[0]', 'x[1]'): 4.000469686132491,
 ('x[3]', 'x[2]'): 4.0001401548903095,
 ('x[1]', 'x[1]'): -5.999953510283767,
 ('x[2]', 'x[2]'): -5.999806412338039}

In [219]:
sampler_dw = DWaveSampler(solver='Advantage_system4.1',token=token)
sampler_qa = EmbeddingComposite(sampler_dw)
start_time = time.time()
sampleset_qa = sampler_qa.sample_qubo(qubo,num_reads=100)
records_qa = sampleset_qa.record
records_qa, time.time() - start_time

(rec.array([([0, 1, 0, 1], -7.99952328, 15, 0.),
            ([0, 0, 1, 1], -7.99939737, 19, 0.),
            ([0, 1, 1, 0], -7.99924636, 21, 0.),
            ([1, 0, 0, 1], -7.99802123, 20, 0.),
            ([1, 1, 0, 0], -7.99787434, 11, 0.),
            ([1, 0, 1, 0], -7.99780338, 14, 0.)],
           dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')]),
 3.9285311698913574)

In [229]:
temp = pd.DataFrame(columns=['Stocks', 'Energy States'])
for r,z in zip(records_qa, lst):
    temp = temp.append({'Stocks':", ".join(z), 'Energy States':r[1]}, ignore_index=True)
temp

,Stocks,Energy States
0,"AAPL, WMT",-7.999523
1,"MSFT, WMT",-7.999397
2,"AAPL, MSFT",-7.999246
3,"AAL, WMT",-7.998021
4,"AAL, AAPL",-7.997874
5,"AAL, MSFT",-7.997803


In [227]:
lst = []
for record in records_qa:
    lst1 = []
    print(record)
    for r, stock in zip(record[0], stocks):
        if r == 1:
            lst1.append(stock)
    lst.append(lst1)
lst

([0, 1, 0, 1], -7.99952328, 15, 0.)
([0, 0, 1, 1], -7.99939737, 19, 0.)
([0, 1, 1, 0], -7.99924636, 21, 0.)
([1, 0, 0, 1], -7.99802123, 20, 0.)
([1, 1, 0, 0], -7.99787434, 11, 0.)
([1, 0, 1, 0], -7.99780338, 14, 0.)


[['AAPL', 'WMT'],
 ['MSFT', 'WMT'],
 ['AAPL', 'MSFT'],
 ['AAL', 'WMT'],
 ['AAL', 'AAPL'],
 ['AAL', 'MSFT']]

Above is the result of 10 samples from the solver. 

First list items are the optimised combination of the stock items, 1 indicates the stock is included in the portfolio. 

The second value is the energy state where the lowest number is the best solution. The number shows close to minus two hundred, because the offset value is not included. 

Third number is the number of occurance of the solution. We have 10 reads and each read has unique stock selections.

The last number indicate "chain break" that the connection between qubits are broken then fixed by the software. Ideal soliution should have 0. chain break ratio.

Using records returned from the D-Wave solver, we can validate if the constraint terms are complied.

In [196]:
portfolio_candidates_qa = []
num_stock_counts_qa = []
record_len = len(records_qa)
for i in range(record_len):
    portfolio = []
    num_stock_count = 0
    for j in range(len(records_qa[i][0])):
        if records_qa[i][0][j] == 1:
            portfolio.append(stocks[j])
            num_stock_count += 1
    portfolio_candidates_qa.append(portfolio)
    num_stock_counts_qa.append(num_stock_count)

In [197]:
num_stock_counts_qa # number of stocks selected for the portfolio

[2, 2, 2, 2]

In our example, 4 of the 10 solutions fit the requirement. We may adjust the coeffient value by increasing the penalty.

Finally, we can select one best solution and display the sotck codes.
We will use 7th record where there was no chain break.

In [50]:
best_portfolio_qa = portfolio_candidates_qa[0]
print(best_portfolio_qa)

['AAPL', 'WMT']


D-Wave has hybrid solver that uses classical computer and quantum annealer. 
The solver 'hybrid_binary_quadratic_model_version2' can have up to 1,000,000 variables.

To try the hybrid solver, we need to import LeapHybridSampler library. The difference from the quantum solver are: 

- it does not require embed composite
- only one sample record is returned (no num_reads parameter)
- it consumes more QPU time (different conversion rate applies QPU: Hybrid 1:20)


In [159]:
from dwave.system import LeapHybridSampler
sampler_dw = DWaveSampler(solver='Advantage_system6.1',token=token)
sampler_qa = EmbeddingComposite(sampler_dw)
sampler_dw_hybrid = LeapHybridSampler(solver='hybrid_binary_quadratic_model_version2',token=token)
sampleset_qa_hybrid = sampler_dw_hybrid.sample_qubo(qubo)
records_qa_hybrid = sampleset_qa_hybrid.record
print(records_qa_hybrid)

[([0, 1, 0, 1], -7.99952364, 1)]


In [52]:
sampleset_qa_hybrid

SampleSet(rec.array([([0, 1, 0, 1], -7.99952364, 1)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['x[0]', 'x[1]', 'x[2]', 'x[3]']), {'qpu_access_time': 127917, 'charge_time': 2995336, 'run_time': 2995336, 'problem_id': '7eb4829c-c872-47ac-ad40-8a1c9b51008f'}, 'BINARY')

In [66]:
records_qa

rec.array([([0, 1, 0, 1], -7.99952328, 5, 0.),
           ([0, 0, 1, 1], -7.99939737, 1, 0.),
           ([0, 1, 1, 0], -7.99924636, 3, 0.),
           ([1, 0, 0, 1], -7.99802123, 1, 0.)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')])

In [55]:
stocks

['AAL', 'AAPL', 'MSFT', 'WMT']

In [67]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
sampleset = sampler.sample_qubo(qubo)

In [73]:
temp = list(sampleset.record)
temp.sort(key = lambda x: x[1])

In [74]:
temp

[([0, 1, 0, 1], -7.99952328, 1),
 ([0, 0, 1, 1], -7.99939737, 1),
 ([0, 1, 1, 0], -7.99924636, 1),
 ([1, 0, 0, 1], -7.99802123, 1),
 ([1, 1, 0, 0], -7.99787434, 1),
 ([1, 0, 1, 0], -7.99780338, 1),
 ([0, 1, 0, 0], -5.99995351, 1),
 ([0, 0, 1, 0], -5.99980641, 1),
 ([0, 0, 0, 1], -5.99973111, 1),
 ([0, 1, 1, 1], -5.99867598, 1),
 ([1, 0, 0, 0], -5.99839052, 1),
 ([1, 1, 0, 1], -5.99734371, 1),
 ([1, 0, 1, 1], -5.99729393, 1),
 ([1, 1, 1, 0], -5.99677364, 1),
 ([0, 0, 0, 0], 0., 1),
 ([1, 1, 1, 1], 0.00389715, 1)]

# 5. Sharpe ratio

In [88]:
import itertools
combinations = []
for combination in itertools.combinations(stocks, 2):
    combinations.append(list(combination))
print(combinations)

[['AAL', 'AAPL'], ['AAL', 'MSFT'], ['AAL', 'WMT'], ['AAPL', 'MSFT'], ['AAPL', 'WMT'], ['MSFT', 'WMT']]


In [97]:
# log daily return
log_return = np.log(data/data.shift(1))

In [99]:
log_return.head()

,AAL,AAPL,MSFT,WMT
Date,,,,
2021-01-04,NaN,NaN,NaN,NaN
2021-01-05,0.019634,0.012288,0.000964,-0.005337
2021-01-06,0.005816,-0.034241,-0.026271,0.006224
2021-01-07,-0.009062,0.033554,0.028060,-0.000068
2021-01-08,-0.016388,0.008594,0.006074,-0.000136


In [100]:
results = {}
for c in combinations:
    print(c)
    weights = np.array(np.random.random(2))
    print('Random Weights:')
    print(weights)

    print('Rebalance')
    weights = weights/np.sum(weights)
    print(weights)

    # expected return
    print('Expected Portfolio Return')
    exp_ret = np.sum((log_return[c].mean()*weights)*252)
    print(exp_ret)

    # expected volatility
    print('Expected Volatility')
    exp_vol = np.sqrt(np.dot(weights.T,np.dot(log_return[c].cov()*252, weights)))
    print(exp_vol)

    # Sharpe Ratio
    print('Sharpe Ratio')
    SR = exp_ret/exp_vol
    print(SR)
    results[", ".join(c)] = SR
    print('**********************')

['AAL', 'AAPL']
Random Weights:
[0.5760542  0.23324352]
Rebalance
[0.71179516 0.28820484]
Expected Portfolio Return
-0.001029363157277835
Expected Volatility
0.4154367007872139
Sharpe Ratio
-0.002477785798239028
**********************
['AAL', 'MSFT']
Random Weights:
[0.105158  0.2187034]
Rebalance
[0.32470063 0.67529937]
Expected Portfolio Return
0.015900120471511334
Expected Volatility
0.2969128496439498
Sharpe Ratio
0.05355147306887643
**********************
['AAL', 'WMT']
Random Weights:
[0.18375207 0.13555124]
Rebalance
[0.57547813 0.42452187]
Expected Portfolio Return
-0.03100821532286579
Expected Volatility
0.3272856822806911
Sharpe Ratio
-0.09474357419727304
**********************
['AAPL', 'MSFT']
Random Weights:
[0.36828281 0.83639026]
Rebalance
[0.30571183 0.69428817]
Expected Portfolio Return
0.06165847820864548
Expected Volatility
0.27395174174229864
Sharpe Ratio
0.2250705829300639
**********************
['AAPL', 'WMT']
Random Weights:
[0.90120443 0.36279611]
Rebalance
[0.71

In [101]:
results

{'AAL, AAPL': -0.002477785798239028,
 'AAL, MSFT': 0.05355147306887643,
 'AAL, WMT': -0.09474357419727304,
 'AAPL, MSFT': 0.2250705829300639,
 'AAPL, WMT': 0.28208935552242476,
 'MSFT, WMT': 0.05085558476454138}

In [107]:
sorted(results.items(), key=lambda x:x[1], reverse=True)

[('AAPL, WMT', 0.28208935552242476),
 ('AAPL, MSFT', 0.2250705829300639),
 ('AAL, MSFT', 0.05355147306887643),
 ('MSFT, WMT', 0.05085558476454138),
 ('AAL, AAPL', -0.002477785798239028),
 ('AAL, WMT', -0.09474357419727304)]

# 6. Monte Carlo

In [231]:
lst = np.array([m for m in list(itertools.product([0, 1], repeat=4)) if sum(m) == 2])
lst

array([[0, 0, 1, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 0, 0]])

In [232]:
cost1 = -np.dot(x, pred_return_rate_mean)

In [233]:
h_cost1

(0.000000 + (-1.000000 * ((Binary('x[3]') * -0.000074) + (Binary('x[2]') * 0.000124) + (Binary('x[0]') * -0.000510) + (Binary('x[1]') * 0.000309))))

In [234]:
cost2 = 0
for i in range(N):
    for j in range(N):
        cost2 += x[i]*x[j]*np.sum((actual_return_rate[i]-actual_return_rate_mean[i])*(actual_return_rate[j]-actual_return_rate_mean[j]))/len(actual_return_rate[i])

In [235]:
cost2

0.001924305369074515

In [236]:
h_cost2

((((Binary('x[3]') * Binary('x[3]')) * 0.089424) * 0.002174) + (((Binary('x[3]') * Binary('x[2]')) * 0.032236) * 0.002174) + (((Binary('x[3]') * Binary('x[1]')) * 0.037109) * 0.002174) + (((Binary('x[3]') * Binary('x[0]')) * 0.023093) * 0.002174) + (((Binary('x[2]') * Binary('x[3]')) * 0.032236) * 0.002174) + (((Binary('x[2]') * Binary('x[2]')) * 0.146064) * 0.002174) + (((Binary('x[2]') * Binary('x[1]')) * 0.118118) * 0.002174) + (((Binary('x[2]') * Binary('x[0]')) * 0.090517) * 0.002174) + (((Binary('x[1]') * Binary('x[3]')) * 0.037109) * 0.002174) + (((Binary('x[1]') * Binary('x[2]')) * 0.118118) * 0.002174) + (((Binary('x[1]') * Binary('x[1]')) * 0.163459) * 0.002174) + (((Binary('x[1]') * Binary('x[0]')) * 0.108028) * 0.002174) + (((Binary('x[0]') * Binary('x[3]')) * 0.023093) * 0.002174) + (((Binary('x[0]') * Binary('x[2]')) * 0.090517) * 0.002174) + (((Binary('x[0]') * Binary('x[1]')) * 0.108028) * 0.002174) + 0.000000 + (((Binary('x[0]') * Binary('x[0]')) * 0.505665) * 0.002174

In [240]:
stock_lst = []
for record in lst:
    lst1 = []
    for r, stock in zip(record, stocks):
        if r == 1:
            lst1.append(stock)
    stock_lst.append(lst1)
stock_lst, lst

([['MSFT', 'WMT'],
  ['AAPL', 'WMT'],
  ['AAPL', 'MSFT'],
  ['AAL', 'WMT'],
  ['AAL', 'MSFT'],
  ['AAL', 'AAPL']],
 array([[0, 0, 1, 1],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 0, 1, 0],
        [1, 1, 0, 0]]))

In [244]:
start_time = time.time()
temp = pd.DataFrame()
for z,x in zip(stock_lst, lst):
    obj = (((((x[3] * x[3]) * 0.089424) * 0.002174) + (((x[3] * x[2]) * 0.032236) * 0.002174) + (((x[3] * x[1]) * 0.037109) * 0.002174) + (((x[3] * x[0]) * 0.023093) * 0.002174) + (((x[2] * x[3]) * 0.032236) * 0.002174) + (((x[2] * x[2]) * 0.146064) * 0.002174) + (((x[2] * x[1]) * 0.118118) * 0.002174) + (((x[2] * x[0]) * 0.090517) * 0.002174) + (((x[1] * x[3]) * 0.037109) * 0.002174) + (((x[1] * x[2]) * 0.118118) * 0.002174) + (((x[1] * x[1]) * 0.163459) * 0.002174) + (((x[1] * x[0]) * 0.108028) * 0.002174) + (((x[0] * x[3]) * 0.023093) * 0.002174) + (((x[0] * x[2]) * 0.090517) * 0.002174) + (((x[0] * x[1]) * 0.108028) * 0.002174) + 0.000000 + (((x[0] * x[0]) * 0.505665) * 0.002174)) + 0.000000 + (-1.000000 * ((x[3] * -0.000074) + (x[2] * 0.000124) + (x[0] * -0.000510) + (x[1] * 0.000309))))
    temp = temp.append({"Stocks": z, "Cost function value": obj}, ignore_index=True)
time.time() - start_time

0.016308069229125977

In [242]:
temp

,combination,objective function value
0,"[MSFT, WMT]",0.000602
1,"[AAPL, WMT]",0.000476
2,"[AAPL, MSFT]",0.000753
3,"[AAL, WMT]",0.001978
4,"[AAL, MSFT]",0.002196
5,"[AAL, AAPL]",0.002125


In [248]:
temp.sort_values("Cost function value", ignore_index=True)

,Stocks,Cost function value
0,"[AAPL, WMT]",0.000476
1,"[MSFT, WMT]",0.000602
2,"[AAPL, MSFT]",0.000753
3,"[AAL, WMT]",0.001978
4,"[AAL, AAPL]",0.002125
5,"[AAL, MSFT]",0.002196
